In [17]:
# if running on cpu
!pip install numpy pillow tqdm sentence_transformers faiss-cpu
# -- OR -- 
# if running on gpu
#!pip install numpy pillow tqdm sentence_transformers faiss-gpu

   ---------------------------------------- 0.0/494.1 kB ? eta -:--:--
   ------------------ --------------------- 225.3/494.1 kB 4.6 MB/s eta 0:00:01
   ----------------------------- ---------- 368.6/494.1 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 494.1/494.1 kB 3.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
   ---------------------------------------- 0.2/18.9 MB 5.9 MB/s eta 0:00:04
    --------------------------------------- 0.4/18.9 MB 4.9 MB/s eta 0:00:04
   - -------------------------------------- 0.6/18.9 MB 5.7 MB/s eta 0:00:04
   - -------------------------------------- 0.8/18.9 MB 5.1 MB/s eta 0:00:04
   - -------------------------------------- 0.9/18.9 MB 4.5 MB/s eta 0:00:04
   -- ------------------------------------- 1.2/18.9 MB 4.8 MB/s eta 0:00:04
   --- ------------------------------------ 1.5/18.9 MB 4.9 MB/s eta 0:00:04
   --- ------------------------------------ 1.7/18.9 MB 4.9 MB/s eta 0:00:04
   

In [21]:
import os, glob, numpy as np
from PIL import Image
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import faiss

'''
This notebook scans images and uses a pretrained CLIP-ViT-B-32 model to create image embeddings, then it creates an 
FAISS (Facebook AI Similarity Search) HNSW (Hierarchical Navigable Small World) index.  It also creates numpy file that
contains the filepaths for the reference images.  The faiss hnsw index file and image filepaths file will be used in the 
next notebook.
'''

REF_IMAGE_DIR = "./reference_images"
INDEX_PATH = "./reference_embeddings/faiss_hnsw.index"
PATHS_NPY  = "./reference_embeddings/ref_image_paths.npy"
BATCH_SIZE = 128
IMG_EXTS   = (".jpg",".jpeg",".png",".webp",".bmp",".tif",".tiff")

# Load the model
model = SentenceTransformer('clip-ViT-B-32')
model.eval()


def load_paths(root):
    return [p for p in glob.glob(os.path.join(root, "**/*"), recursive=True)
            if p.lower().endswith(IMG_EXTS)]


def pil_open_rgb(p):
    try:
        return Image.open(p).convert("RGB")
    except:
        return None

paths = load_paths(REF_IMAGE_DIR)
print("Reference images:", len(paths))
np.save(PATHS_NPY, np.array(paths, dtype=object))

# model = SentenceTransformer(MODEL_NAME)
dim = 512  # ViT-B/32
# HNSW with cosine (inner product on normalized vectors)
index = faiss.IndexHNSWFlat(dim, 32, faiss.METRIC_INNER_PRODUCT)
index.hnsw.efConstruction = 200

# Encode in deterministic order and add in the same order
batch_imgs, batch_ids = [], []
added = 0

for rid, p in enumerate(tqdm(paths, desc="Embedding+Add")):
    img = pil_open_rgb(p)
    if img is None:
        # Skip unreadable images to avoid zero vectors corrupting the space
        continue
    batch_imgs.append(img)
    batch_ids.append(rid)

    if len(batch_imgs) == BATCH_SIZE:
        # batch_imgs can be a list of PIL Images, NumPy arrays, or image file paths
        vecs = (model.encode(
                    batch_imgs,
                    convert_to_numpy=True,
                    normalize_embeddings=True,  # L2 normalize
                    show_progress_bar=False,
                    batch_size=32,  # adjust to your GPU/CPU memory
                    ).astype("float32"))
        # vecs = model.encode(batch_imgs, convert_to_numpy=True, normalize_embeddings=True).astype("float32")
        index.add(vecs)  # vectors are normalized; IP acts as cosine
        batch_imgs.clear(); batch_ids.clear()
        added += len(vecs)

if batch_imgs:
    vecs = model.encode(batch_imgs, convert_to_numpy=True, normalize_embeddings=True).astype("float32")
    index.add(vecs)
    added += len(vecs)

faiss.write_index(index, INDEX_PATH)
print("Wrote index:", INDEX_PATH, "vectors:", added)


Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

CLIPModel LOAD REPORT from: C:\Users\tev\.cache\huggingface\hub\models--sentence-transformers--clip-ViT-B-32\snapshots\327ab6726d33c0e22f920c83f2ff9e4bd38ca37f\0_CLIPModel
Key                                  | Status     |  | 
-------------------------------------+------------+--+-
vision_model.embeddings.position_ids | UNEXPECTED |  | 
text_model.embeddings.position_ids   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Reference images: 20


Embedding+Add: 100%|███████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 57.72it/s]


Wrote index: ./reference_embeddings/faiss_hnsw.index vectors: 20
